In [1]:
!git clone https://github.com/pthengtr/kcw-analytics.git

Cloning into 'kcw-analytics'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 105 (delta 45), reused 68 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (105/105), 65.14 KiB | 1.07 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [2]:
!cd /content/kcw-analytics && git pull origin main

From https://github.com/pthengtr/kcw-analytics
 * branch            main       -> FETCH_HEAD
Already up to date.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import pandas as pd

folder = "/content/drive/MyDrive/kcw_analytics/01_raw"

data = {}

for file in os.listdir(folder):
    if file.endswith(".csv"):
        path = os.path.join(folder, file)
        data[file] = pd.read_csv(
            path,
            dtype={
              "BCODE": "string",
              "ITEMNO": "string",
              "BILLNO": "string",
            },
            encoding="utf-8-sig",
            low_memory=False   # stops chunk guessing
        )
        print(f"Loaded: {file} -> {data[file].shape}")

Loaded: raw_hq_pimas_purchase_bills.csv -> (82716, 49)
Loaded: raw_hq_simas_sales_bills.csv -> (481937, 49)
Loaded: raw_hq_pidet_purchase_lines.csv -> (246580, 41)
Loaded: raw_hq_sidet_sales_lines.csv -> (1187205, 38)
Loaded: raw_hq_icmas_products.csv -> (114755, 94)
Loaded: raw_syp_pimas_purchase_bills.csv -> (2721, 49)
Loaded: raw_syp_simas_sales_bills.csv -> (10368, 49)
Loaded: raw_syp_sidet_sales_lines.csv -> (30374, 38)
Loaded: raw_syp_pidet_purchase_lines.csv -> (25555, 41)
Loaded: raw_inventory_hq_2024.csv -> (4983, 8)


In [5]:
import sys
import importlib

# ensure repo is on path
repo_path = "/content/kcw-analytics"
if repo_path not in sys.path:
    sys.path.append(repo_path)

# import the module (NOT individual functions)
import src.kcw.utils as utils

# reload to pick up latest .py changes
importlib.reload(utils)

get_vat_sales_lines_last_purchase_nonvat = utils.get_vat_sales_lines_last_purchase_nonvat
audit_bcode_vat_sales_last_purchase = utils.audit_bcode_vat_sales_last_purchase

In [6]:
YEAR = 2026

vat_sales_last_purchase_nonvat_hq = get_vat_sales_lines_last_purchase_nonvat(
    data, year=YEAR, source="hq"
)

vat_sales_last_purchase_nonvat_syp = get_vat_sales_lines_last_purchase_nonvat(
    data, year=YEAR, source="syp"
)


In [7]:
vat_sales_last_purchase_nonvat_hq

,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,DED,VAT,AMOUNT,CHGAMT,ACCTNO,PAID,ACCT_NO,DONE,CANCELED,LAST_PURCHASE_ISVAT
23,1226927,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-011,10,1,12050690,...,NaN,7.0,590.0,NaN,7LPNT,N,CRRF,N,N,N
24,1226858,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-004,10,1,12050729,...,NaN,7.0,350.0,NaN,7SPNT,N,BM,N,N,N
25,1227016,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-013,10,1,12051272,...,NaN,7.0,205.0,NaN,7SPNT,N,KCY,N,N,N
27,1226983,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-012,20,2,12051561,...,NaN,7.0,140.0,NaN,7ICE,N,BM,N,N,N
28,1226984,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-012,30,3,12051563,...,NaN,7.0,90.0,NaN,7ICE,N,BM,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,1240464,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TD6901-098,10,1,30050835,...,NaN,7.0,90.0,NaN,7TET,N,CRRK,N,N,N
5096,1240465,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TD6901-099,10,1,30050835,...,NaN,7.0,90.0,NaN,7TET,N,CRRK,N,N,N
5097,1240467,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TD6901-100,10,1,30050835,...,NaN,7.0,90.0,NaN,7TET,N,CRRK,N,N,N
5099,1240501,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TAD6901-535,20,2,30051639,...,NaN,7.0,490.0,NaN,7LAZ1,N,CRRK,N,N,N


In [8]:
vat_sales_last_purchase_nonvat_hq_tar_only = vat_sales_last_purchase_nonvat_hq[
    vat_sales_last_purchase_nonvat_hq["BILLNO"].str.match(r"^TAR", na=False)
]
vat_sales_last_purchase_nonvat_hq_tar_only

,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,DED,VAT,AMOUNT,CHGAMT,ACCTNO,PAID,ACCT_NO,DONE,CANCELED,LAST_PURCHASE_ISVAT
1191,1230247,0,SJ,2026-01-07 00:00:00,1.0,2026-01-07,TAR6901-104,10,1,13026861,...,NaN,7.0,40.0,NaN,7000,N,NaN,N,N,N
1409,1231020,0,SJ,2026-01-08 00:00:00,1.0,2026-01-08,TAR6901-134,10,1,12053734,...,NaN,7.0,7920.0,NaN,7000,N,NaN,N,N,N
2268,1233212,0,SJ,2026-01-12 00:00:00,1.0,2026-01-12,TAR6901-191,40,4,19040111,...,NaN,7.0,20.0,NaN,7000,N,7ANP,N,N,N
2970,1235322,0,SJ,2026-01-16 00:00:00,1.0,2026-01-16,TAR6901-255,50,5,12051662,...,NaN,7.0,150.0,NaN,7000,N,NaN,N,N,N
3650,1237036,0,SJ,2026-01-19 00:00:00,1.0,2026-01-19,TAR6901-309,10,1,12053734,...,NaN,7.0,5940.0,NaN,7000,N,NaN,N,N,N


In [9]:
vat_sales_last_purchase_nonvat_syp

,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,DED,VAT,AMOUNT,CHGAMT,ACCTNO,PAID,ACCT_NO,DONE,CANCELED,LAST_PURCHASE_ISVAT
453,1693718,1,SJ,2026-01-08 00:00:00,1,2026-01-08,3TR6901-005,30,3,13036340,...,NaN,7.0,9.0,NaN,7ชจร,N,LK,N,N,N
648,1744098,0,SJ,2026-01-12 00:00:00,1,2026-01-12,3TAR6901-072,70,7,13027010,...,NaN,7.0,6.8,NaN,7000,N,LK,N,N,N
713,1754135,1,SJ,2026-01-13 00:00:00,1,2026-01-13,3TR6901-008,20,2,09011683,...,NaN,7.0,430.0,NaN,7อลย,N,SRI,N,N,N
878,1774163,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,20,2,05018690,...,NaN,7.0,60.0,NaN,7ชจร,N,SSWK,N,N,N
888,1774167,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,60,6,10050185,...,NaN,7.0,108.0,NaN,7ชจร,N,YOKO,N,N,N
889,1774162,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,10,1,12050484,...,NaN,7.0,240.0,NaN,7ชจร,N,SJP,N,N,N
1048,1774591,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,60,6,14010442,...,NaN,7.0,60.0,NaN,7ชจร,N,RJS,N,N,N
1053,1774588,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,40,4,17016070,...,NaN,7.0,160.0,NaN,7ชจร,N,LK,N,N,N
1089,1774587,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,30,3,33010369,...,NaN,7.0,200.0,NaN,7ชจร,N,MC,N,N,N
1123,1784711,1,SJ,2026-01-19 00:00:00,1,2026-01-19,3TR6901-013,60,6,13051791,...,NaN,7.0,28.0,NaN,7ชจร,N,SMA,N,N,N


In [10]:
vat_sales_last_purchase_nonvat_syp_tar_only = vat_sales_last_purchase_nonvat_syp[
    vat_sales_last_purchase_nonvat_syp["BILLNO"].str.match(r"^3TAR", na=False)
]
vat_sales_last_purchase_nonvat_syp_tar_only

,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,DED,VAT,AMOUNT,CHGAMT,ACCTNO,PAID,ACCT_NO,DONE,CANCELED,LAST_PURCHASE_ISVAT
648,1744098,0,SJ,2026-01-12 00:00:00,1,2026-01-12,3TAR6901-072,70,7,13027010,...,NaN,7.0,6.8,NaN,7000,N,LK,N,N,N
1334,1805220,0,SJ,2026-01-22 00:00:00,1,2026-01-22,3TAR6901-149,40,4,12052373,...,NaN,7.0,620.0,NaN,7000,N,NaN,N,N,N
1416,1815285,0,SJ,2026-01-23 00:00:00,1,2026-01-23,3TAR6901-154,10,1,12052692,...,NaN,7.0,750.0,NaN,7000,N,7CNH,N,N,N
1468,1815286,0,SJ,2026-01-23 00:00:00,1,2026-01-23,3TAR6901-154,20,2,32050343,...,NaN,7.0,1650.0,NaN,7000,N,KT,N,N,N
1480,1825359,0,SJ,2026-01-24 00:00:00,1,2026-01-24,3TAR6901-165,20,2,05057622,...,NaN,7.0,320.0,NaN,7000,N,NaN,N,N,NaN
1487,1825320,0,SJ,2026-01-24 00:00:00,1,2026-01-24,3TAR6901-161,30,3,13026790,...,NaN,7.0,40.0,NaN,7000,N,YSR,N,N,N
1531,1825352,0,SJ,2026-01-24 00:00:00,1,2026-01-24,3TAR6901-164,50,5,32050343,...,NaN,7.0,1650.0,NaN,7000,N,NaN,N,N,N


In [11]:
vat_sales_last_purchase_nonvat_hq_cleaned = vat_sales_last_purchase_nonvat_hq[
    ~vat_sales_last_purchase_nonvat_hq["BILLNO"]
        .astype(str)
        .str.startswith("TAR", na=False)
]

vat_sales_last_purchase_nonvat_hq_cleaned

,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,DED,VAT,AMOUNT,CHGAMT,ACCTNO,PAID,ACCT_NO,DONE,CANCELED,LAST_PURCHASE_ISVAT
23,1226927,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-011,10,1,12050690,...,NaN,7.0,590.0,NaN,7LPNT,N,CRRF,N,N,N
24,1226858,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-004,10,1,12050729,...,NaN,7.0,350.0,NaN,7SPNT,N,BM,N,N,N
25,1227016,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-013,10,1,12051272,...,NaN,7.0,205.0,NaN,7SPNT,N,KCY,N,N,N
27,1226983,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-012,20,2,12051561,...,NaN,7.0,140.0,NaN,7ICE,N,BM,N,N,N
28,1226984,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-012,30,3,12051563,...,NaN,7.0,90.0,NaN,7ICE,N,BM,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,1240464,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TD6901-098,10,1,30050835,...,NaN,7.0,90.0,NaN,7TET,N,CRRK,N,N,N
5096,1240465,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TD6901-099,10,1,30050835,...,NaN,7.0,90.0,NaN,7TET,N,CRRK,N,N,N
5097,1240467,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TD6901-100,10,1,30050835,...,NaN,7.0,90.0,NaN,7TET,N,CRRK,N,N,N
5099,1240501,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TAD6901-535,20,2,30051639,...,NaN,7.0,490.0,NaN,7LAZ1,N,CRRK,N,N,N


In [12]:
vat_sales_last_purchase_nonvat_syp_cleaned = vat_sales_last_purchase_nonvat_syp[
    ~vat_sales_last_purchase_nonvat_syp["BILLNO"]
        .astype(str)
        .str.startswith("3TAR", na=False)
]

vat_sales_last_purchase_nonvat_syp_cleaned

,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,DED,VAT,AMOUNT,CHGAMT,ACCTNO,PAID,ACCT_NO,DONE,CANCELED,LAST_PURCHASE_ISVAT
453,1693718,1,SJ,2026-01-08 00:00:00,1,2026-01-08,3TR6901-005,30,3,13036340,...,NaN,7.0,9.0,NaN,7ชจร,N,LK,N,N,N
713,1754135,1,SJ,2026-01-13 00:00:00,1,2026-01-13,3TR6901-008,20,2,09011683,...,NaN,7.0,430.0,NaN,7อลย,N,SRI,N,N,N
878,1774163,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,20,2,05018690,...,NaN,7.0,60.0,NaN,7ชจร,N,SSWK,N,N,N
888,1774167,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,60,6,10050185,...,NaN,7.0,108.0,NaN,7ชจร,N,YOKO,N,N,N
889,1774162,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,10,1,12050484,...,NaN,7.0,240.0,NaN,7ชจร,N,SJP,N,N,N
1048,1774591,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,60,6,14010442,...,NaN,7.0,60.0,NaN,7ชจร,N,RJS,N,N,N
1053,1774588,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,40,4,17016070,...,NaN,7.0,160.0,NaN,7ชจร,N,LK,N,N,N
1089,1774587,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,30,3,33010369,...,NaN,7.0,200.0,NaN,7ชจร,N,MC,N,N,N
1123,1784711,1,SJ,2026-01-19 00:00:00,1,2026-01-19,3TR6901-013,60,6,13051791,...,NaN,7.0,28.0,NaN,7ชจร,N,SMA,N,N,N
1414,1815252,1,SJ,2026-01-23 00:00:00,1,2026-01-23,3TR6901-015,20,2,12051665,...,NaN,7.0,30.0,NaN,7ชจร,N,SMA,N,N,N


In [13]:
#sales_one, pidet_one = audit_bcode_vat_sales_last_purchase(data, "12050690", YEAR, "hq")

In [14]:
pidet = data["raw_hq_pidet_purchase_lines.csv"].copy()
pidet_filtered = pidet[pidet["BCODE"] == "13036340"]

pidet_filtered[['QTY']]

,QTY
2496,1500.0
54555,1500.0
112707,1500.0
149551,1500.0
195081,1500.0


In [15]:
vat_sales_last_purchase_nonvat_hq_cleaned[['BCODE', 'BILLNO', 'QTY', 'MTP', 'UI', 'AMOUNT', 'ACCT_NO', 'CANCELED','ISVAT', 'LAST_PURCHASE_ISVAT']]

,BCODE,BILLNO,QTY,MTP,UI,AMOUNT,ACCT_NO,CANCELED,ISVAT,LAST_PURCHASE_ISVAT
23,12050690,TAD6901-011,1.0,1.0,หน่วย,590.0,CRRF,N,Y,N
24,12050729,TAD6901-004,1.0,1.0,หน่วย,350.0,BM,N,Y,N
25,12051272,TAD6901-013,1.0,1.0,ชุด,205.0,KCY,N,Y,N
27,12051561,TAD6901-012,2.0,1.0,หน่วย,140.0,BM,N,Y,N
28,12051563,TAD6901-012,2.0,1.0,หน่วย,90.0,BM,N,Y,N
...,...,...,...,...,...,...,...,...,...,...
5095,30050835,TD6901-098,1.0,1.0,หน่วย,90.0,CRRK,N,Y,N
5096,30050835,TD6901-099,1.0,1.0,หน่วย,90.0,CRRK,N,Y,N
5097,30050835,TD6901-100,1.0,1.0,หน่วย,90.0,CRRK,N,Y,N
5099,30051639,TAD6901-535,2.0,1.0,เส้น,490.0,CRRK,N,Y,N


In [16]:
vat_sales_last_purchase_nonvat_syp_cleaned

,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,DED,VAT,AMOUNT,CHGAMT,ACCTNO,PAID,ACCT_NO,DONE,CANCELED,LAST_PURCHASE_ISVAT
453,1693718,1,SJ,2026-01-08 00:00:00,1,2026-01-08,3TR6901-005,30,3,13036340,...,NaN,7.0,9.0,NaN,7ชจร,N,LK,N,N,N
713,1754135,1,SJ,2026-01-13 00:00:00,1,2026-01-13,3TR6901-008,20,2,09011683,...,NaN,7.0,430.0,NaN,7อลย,N,SRI,N,N,N
878,1774163,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,20,2,05018690,...,NaN,7.0,60.0,NaN,7ชจร,N,SSWK,N,N,N
888,1774167,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,60,6,10050185,...,NaN,7.0,108.0,NaN,7ชจร,N,YOKO,N,N,N
889,1774162,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,10,1,12050484,...,NaN,7.0,240.0,NaN,7ชจร,N,SJP,N,N,N
1048,1774591,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,60,6,14010442,...,NaN,7.0,60.0,NaN,7ชจร,N,RJS,N,N,N
1053,1774588,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,40,4,17016070,...,NaN,7.0,160.0,NaN,7ชจร,N,LK,N,N,N
1089,1774587,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,30,3,33010369,...,NaN,7.0,200.0,NaN,7ชจร,N,MC,N,N,N
1123,1784711,1,SJ,2026-01-19 00:00:00,1,2026-01-19,3TR6901-013,60,6,13051791,...,NaN,7.0,28.0,NaN,7ชจร,N,SMA,N,N,N
1414,1815252,1,SJ,2026-01-23 00:00:00,1,2026-01-23,3TR6901-015,20,2,12051665,...,NaN,7.0,30.0,NaN,7ชจร,N,SMA,N,N,N


In [36]:
import pandas as pd
import numpy as np

def enrich_sales_with_avcost_and_newbillno(
    data: dict,
    sales_lines: pd.DataFrame,
    *,
    source: str,  # <-- NEW: "hq" or "syp"
    purchase_key: str = "raw_hq_pidet_purchase_lines.csv",
    # common columns
    bcode_col: str = "BCODE",
    date_col: str = "BILLDATE",
    # sales qty columns (kept for signature compatibility)
    sales_qty_col: str = "QTY",
    sales_mtp_col: str = "MTP",
    # purchase columns for cost calc
    purchase_amount_col: str = "AMOUNT",
    purchase_qty_col: str = "QTY",
    purchase_mtp_col: str = "MTP",
    # bill cols (kept for signature compatibility)
    billno_col: str = "BILLNO",
    # output
    output_cols: tuple = (
        "BCODE", "BILLDATE", "BILLNO", "QTY", "MTP", "UI", "AMOUNT", "ACCT_NO",
        "CANCELED", "ISVAT", "LAST_PURCHASE_ISVAT", "AV_COST"
    ),
) -> pd.DataFrame:
    """
    Takes already-filtered sales lines and:
      1) Looks up weighted-average UNIT cost as-of sale date from purchases in data[purchase_key].
      2) Adds NEW_BILLNO:
         - HQ  -> RV[BDyy][mm]-[seq]
         - SYP -> 3RV[BDyy][mm]-[seq]
         - Sequence resets per MONTH
         - Within a month, sequence increments by 1 for each new DATE,
           and for each additional chunk of 10 lines on the same DATE.
      3) Returns only requested columns (plus NEW_BILLNO).
    """

    # --- validate source ---
    source = source.lower().strip()
    if source not in ("hq", "syp"):
        raise ValueError("source must be 'hq' or 'syp'")

    bill_prefix = "RV" if source == "hq" else "3RV"

    # --- helpers ---
    def _clean_cols(df: pd.DataFrame) -> pd.DataFrame:
        out = df.copy()
        out.columns = (
            out.columns.astype(str)
            .str.replace("\ufeff", "", regex=False)
            .str.strip()
        )
        return out

    def _clean_bcode(s: pd.Series) -> pd.Series:
        return s.astype(str).str.strip()

    # --- copy & clean ---
    sales = _clean_cols(sales_lines).copy()
    purch = _clean_cols(data[purchase_key]).copy()

    # Ensure key columns exist
    for col in [bcode_col, date_col]:
        if col not in sales.columns:
            raise KeyError(f"sales_lines missing required column: {col}")
        if col not in purch.columns:
            raise KeyError(f"purchase data missing required column: {col}")

    # Clean BCODE + dates
    sales[bcode_col] = _clean_bcode(sales[bcode_col])
    purch[bcode_col] = _clean_bcode(purch[bcode_col])

    sales[date_col] = pd.to_datetime(sales[date_col], errors="coerce")
    purch[date_col] = pd.to_datetime(purch[date_col], errors="coerce")

    sales = sales.dropna(subset=[bcode_col, date_col]).copy()
    purch = purch.dropna(subset=[bcode_col, date_col]).copy()

    sales = sales[sales[bcode_col] != ""].copy()
    purch = purch[purch[bcode_col] != ""].copy()

    # Reduce purchases to only needed BCODEs (perf)
    purch = purch[purch[bcode_col].isin(sales[bcode_col].unique())].copy()

    # --- 1) AV_COST as-of sale date ---
    for c in [purchase_amount_col, purchase_qty_col, purchase_mtp_col]:
        purch[c] = pd.to_numeric(purch[c], errors="coerce")

    purch = purch.dropna(subset=[purchase_amount_col, purchase_qty_col, purchase_mtp_col]).copy()
    purch = purch[(purch[purchase_qty_col] != 0) & (purch[purchase_mtp_col] > 0)].copy()

    purch["__COST_BASIS__"] = (
        purch[purchase_amount_col] / purch[purchase_qty_col]
    ) * purch[purchase_mtp_col]

    purch = purch.sort_values([bcode_col, date_col], kind="mergesort")
    purch["__CUM_COST__"] = purch.groupby(bcode_col)["__COST_BASIS__"].cumsum()
    purch["__CUM_MTP__"] = purch.groupby(bcode_col)[purchase_mtp_col].cumsum()
    purch["__AVCOST__"] = purch["__CUM_COST__"] / purch["__CUM_MTP__"]

    purch_key = purch[[bcode_col, date_col, "__AVCOST__"]]

    sales_sorted = sales.sort_values([date_col, bcode_col], kind="mergesort").reset_index(drop=True)
    purch_key = purch_key.sort_values([date_col, bcode_col], kind="mergesort").reset_index(drop=True)

    merged = pd.merge_asof(
        sales_sorted,
        purch_key,
        left_on=date_col,
        right_on=date_col,
        by=bcode_col,
        direction="backward",
        allow_exact_matches=True,
    )

    merged["AV_COST"] = merged["__AVCOST__"]
    merged.drop(columns="__AVCOST__", inplace=True)

    # --- 2) NEW_BILLNO (monthly reset, daily + 10-line chunk increment) ---
    merged["__DATEKEY__"] = merged[date_col].dt.normalize()
    merged["__MONTHKEY__"] = merged["__DATEKEY__"].dt.to_period("M")

    merged = merged.sort_values(["__DATEKEY__", bcode_col], kind="mergesort").reset_index(drop=True)

    day_chunk = merged.groupby("__DATEKEY__", sort=False).cumcount() // 10

    day_tbl = (
        pd.DataFrame({
            "__MONTHKEY__": merged["__MONTHKEY__"],
            "__DATEKEY__": merged["__DATEKEY__"],
            "day_chunk": day_chunk
        })
        .groupby(["__MONTHKEY__", "__DATEKEY__"], sort=False)["day_chunk"]
        .max()
        .rename("MAX_CHUNK")
        .reset_index()
        .sort_values(["__MONTHKEY__", "__DATEKEY__"], kind="mergesort")
    )

    day_tbl["DAY_START_SEQ0"] = (
        day_tbl.groupby("__MONTHKEY__")["MAX_CHUNK"]
        .transform(lambda x: (x + 1).cumsum() - (x + 1))
    )

    start_map = day_tbl.set_index(["__MONTHKEY__", "__DATEKEY__"])["DAY_START_SEQ0"]
    merged["__DAY_START_SEQ0__"] = start_map.loc[
        list(zip(merged["__MONTHKEY__"], merged["__DATEKEY__"]))
    ].to_numpy()

    seq = (merged["__DAY_START_SEQ0__"] + day_chunk + 1).astype(int)

    bd_yy = ((merged["__DATEKEY__"].dt.year + 543) % 100).astype(int)
    mm = merged["__DATEKEY__"].dt.month.astype(int)

    merged["NEW_BILLNO"] = (
        bill_prefix
        + bd_yy.map(lambda x: f"{x:02d}")
        + mm.map(lambda x: f"{x:02d}")
        + "-"
        + seq.map(lambda x: f"{x:03d}")
    )

    merged.drop(columns=["__DATEKEY__", "__MONTHKEY__", "__DAY_START_SEQ0__"], inplace=True)

    # --- 3) Output ---
    for col in output_cols:
        if col not in merged.columns:
            merged[col] = pd.NA

    return merged[list(output_cols) + ["NEW_BILLNO"]].copy()


In [39]:
out_syp = enrich_sales_with_avcost_and_newbillno(
    data=data,
    sales_lines=vat_sales_last_purchase_nonvat_syp_cleaned,
    source = "syp"
)

out_hq = enrich_sales_with_avcost_and_newbillno(
    data=data,
    sales_lines=vat_sales_last_purchase_nonvat_hq_cleaned,
    source = "hq"
)


In [40]:
out_syp

,BCODE,BILLDATE,BILLNO,QTY,MTP,UI,AMOUNT,ACCT_NO,CANCELED,ISVAT,LAST_PURCHASE_ISVAT,AV_COST,NEW_BILLNO
0,13036340,2026-01-08,3TR6901-005,6.0,1.0,ตัว,9.0,LK,N,Y,N,0.175388,3RV6901-001
1,09011683,2026-01-13,3TR6901-008,1.0,1.0,หน่วย,430.0,SRI,N,Y,N,350.000000,3RV6901-002
2,05018690,2026-01-15,3TR6901-010,3.0,1.0,ตัว,60.0,SSWK,N,Y,N,7.769231,3RV6901-003
3,10050185,2026-01-15,3TR6901-010,6.0,1.0,ตัว,108.0,YOKO,N,Y,N,9.590909,3RV6901-003
4,12050484,2026-01-15,3TR6901-010,12.0,1.0,หน่วย,240.0,SJP,N,Y,N,9.761765,3RV6901-003
5,14010442,2026-01-17,3TR6901-012,2.0,1.0,เส้น,60.0,RJS,N,Y,N,2318.911359,3RV6901-004
6,17016070,2026-01-17,3TR6901-012,8.0,1.0,ตัว,160.0,LK,N,Y,N,10.011600,3RV6901-004
7,33010369,2026-01-17,3TR6901-012,1.0,1.0,ชุด,200.0,MC,N,Y,N,101.944444,3RV6901-004
8,13051791,2026-01-19,3TR6901-013,1.0,1.0,หัว,28.0,SMA,N,Y,N,14.000000,3RV6901-005
9,12051665,2026-01-23,3TR6901-015,2.0,1.0,หน่วย,30.0,SMA,N,Y,N,5.833333,3RV6901-006


In [41]:
out_hq

,BCODE,BILLDATE,BILLNO,QTY,MTP,UI,AMOUNT,ACCT_NO,CANCELED,ISVAT,LAST_PURCHASE_ISVAT,AV_COST,NEW_BILLNO
0,12050690,2026-01-01,TAD6901-011,1.0,1.0,หน่วย,590.0,CRRF,N,Y,N,415.517241,RV6901-001
1,12050729,2026-01-01,TAD6901-004,1.0,1.0,หน่วย,350.0,BM,N,Y,N,207.142857,RV6901-001
2,12051272,2026-01-01,TAD6901-013,1.0,1.0,ชุด,205.0,KCY,N,Y,N,95.625000,RV6901-001
3,12051561,2026-01-01,TAD6901-012,2.0,1.0,หน่วย,140.0,BM,N,Y,N,35.502002,RV6901-001
4,12051563,2026-01-01,TAD6901-012,2.0,1.0,หน่วย,90.0,BM,N,Y,N,16.435781,RV6901-001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,30050835,2026-01-26,TD6901-098,1.0,1.0,หน่วย,90.0,CRRK,N,Y,N,62.702128,RV6901-052
426,30050835,2026-01-26,TD6901-099,1.0,1.0,หน่วย,90.0,CRRK,N,Y,N,62.702128,RV6901-052
427,30050835,2026-01-26,TD6901-100,1.0,1.0,หน่วย,90.0,CRRK,N,Y,N,62.702128,RV6901-052
428,30051639,2026-01-26,TAD6901-535,2.0,1.0,เส้น,490.0,CRRK,N,Y,N,80.657895,RV6901-052
